# 药品销售市场数据分析  

### 数据说明  

数据集为 [北京朝阳某医院2018年的药品销售数据](https://www.heywhale.com/mw/dataset/5e4a964580da780037bc664e)，其中包含7个字段信息：  
1、购药时间  
2、社保卡号  
3、商品编码  
4、商品名称  
5、销售数量  
6、应收金额  
7、实收金额  


**销售数据分析**主要是指整体上销售的情况：  
1. 卖的最好/最差的药品是？  
2. 整体销售额的时间变化趋势？  
3. 各月份卖的最好的药品是那些  

在进行以上分析时，需要自行判断评定指标，如在分析**卖的最好的药品**时，如何评定**最好**这一概念，是销售额最高即最好还是利润最大为最好。  

进行整体的分析之后我们可以进行**相关性分析**。  

在本例中，**相关性分析**是指分析哪些特征影响着药品的销售。结合数据集所提供的字段，可以进行以下的分析：  

1. 购药时间是否影响药品的销售情况（这里的时间可以指**工作日与否**或是**某个月份**）  
2. 人们是否会更愿意购买可使用社保减免的药品？  

通过上述的分析，我们可以确定影响药品销售的因素有哪些，为医院的下一次药品采购提供合理的采购建议！

# 1、Python库导入与数据读取  

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 记得补充你需要用到的Python包

df = pd.read_excel("/home/mw/input/medical9242/朝阳医院2018年销售数据.xlsx")

# 2、数据预览与预处理

In [2]:
# 查看数据维度
df.shape

(6578, 7)

In [3]:
# 查看数据信息  
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6578 entries, 0 to 6577
Data columns (total 7 columns):
购药时间    6576 non-null object
社保卡号    6576 non-null float64
商品编码    6577 non-null float64
商品名称    6577 non-null object
销售数量    6577 non-null float64
应收金额    6577 non-null float64
实收金额    6577 non-null float64
dtypes: float64(5), object(2)
memory usage: 359.8+ KB


In [4]:
# 查看各列的缺失值
df.isna().sum()

购药时间    2
社保卡号    2
商品编码    1
商品名称    1
销售数量    1
应收金额    1
实收金额    1
dtype: int64

In [5]:
# 缺失值处理
# 因为缺失值比较少，所以直接删除
df.dropna(axis=0,inplace=True) 

In [6]:
# 查看重复值数量
df.duplicated().sum()

# 删除重复值
df.drop_duplicates(inplace=True)

In [7]:
#通过上述的数据探索，发现“购药时间”、“社保卡号”、“商品编码”、“销售数量”的格式需要改变
df["购药日期"], df["星期"] = df["购药时间"].str.split(' ', 1).str
df["购药日期"] = df["购药日期"].replace("2018-02-29", "2018-02-28") #因为2018-2-29不存在，所以把它替换成2018-02-28
df["购药日期"] = pd.to_datetime(df["购药日期"])
df.drop(columns=["购药时间"], inplace=True)
df["社保卡号"] = df["社保卡号"].astype(str)
df["商品编码"] = df["商品编码"].astype(str)
df["社保卡号"] = df["社保卡号"].str.split('.').str[0]
df["商品编码"] = df["商品编码"].str.split('.').str[0]
df["销售数量"] = df["销售数量"].astype(int)

# 3、数据分析

In [8]:
#回顾下数据集
df.head()

,社保卡号,商品编码,商品名称,销售数量,应收金额,实收金额,购药日期,星期
0,1616528,236701,强力VC银翘片,6,82.8,69.00,2018-01-01,星期五
1,1616528,236701,清热解毒口服液,1,28.0,24.64,2018-01-02,星期六
2,12602828,236701,感康,2,16.8,15.00,2018-01-06,星期三
3,10070343428,236701,三九感冒灵,1,28.0,28.00,2018-01-11,星期一
4,101554328,236701,三九感冒灵,8,224.0,208.00,2018-01-15,星期五


### 任务1：销售情况最好的药品是？最差的药品是？  

提示：通过对药品进行分组聚合统计，在此处我们评定销售情况好的标准是销售数量总和。之后取出前n个药品进行可视化展示

In [9]:
grouped_df = df.groupby("商品名称").agg({"销售数量": "sum"}).sort_values(by="销售数量", ascending=False)
top_10_medicines = grouped_df.head(10) #销售前十的药品
bottom_10_medicines = grouped_df.tail(10) #销售倒十的药品
print(f'销售前十的药品及销售数量:\n{top_10_medicines}')
print('-'*100)
print(f'销售倒十的药品及销售数量:\n{bottom_10_medicines}')

销售前十的药品及销售数量:
                   销售数量
商品名称                   
苯磺酸氨氯地平片(安内真)      1785
开博通                1458
酒石酸美托洛尔片(倍他乐克)     1138
硝苯地平片(心痛定)          825
苯磺酸氨氯地平片(络活喜)       796
复方利血平片(复方降压片)       517
G琥珀酸美托洛尔缓释片(倍他乐克)   509
缬沙坦胶囊(代文)           444
非洛地平缓释片(波依定)        375
高特灵                 372
----------------------------------------------------------------------------------------------------
销售倒十的药品及销售数量:
                     销售数量
商品名称                     
G厄贝沙坦分散片(6盒/疗程)        18
**阿替洛尔片                17
G硝苯地平缓释片(II)(6盒/疗程)    16
苯磺酸氨氯地平片(兰迪)           14
培哚普利片(雅施达)             10
D替格瑞洛片                 10
赖诺普利片(信赖安)              5
TG盐酸贝那普利片(新亚富舒)         2
TG厄贝沙坦片                 2
D厄贝沙坦氢氯噻嗪片(倍悦)          2


In [10]:
#绘制销售排名前10的药品条形图
plt.figure(figsize=(12, 8))
sns.barplot(y=top_10_medicines.index, x=top_10_medicines["销售数量"], palette="viridis")
plt.title("销售排名前10的药品条形图")
plt.xlabel("总销售量")
plt.ylabel("药品名称")
plt.show()

<Figure size 864x576 with 1 Axes>

#### 销售情况最好的药品是：苯磺酸氨氯地平片(安内真)，销售数量：**1785**  
##### 销售情况最差的药品是：D厄贝沙坦氢氯噻嗪片(倍悦)、TG厄贝沙坦片、TG盐酸贝那普利片(新亚富舒)，销售数量：2

### 任务2：整体销售额的时间变化趋势  

提示：  

数据集中存在时间数据，但是不够简洁，所以需要先对时间数据进行处理：  
```
df["购药时间_月"] = df["购药时间"].map(lambda x:x.split(" ")[0].split("-"[1])  ##购药日期所属月份  
df["购药时间_星期"] = df["购药时间"].map(lambda x:x.split(" ")[1])  ##购药当天是星期几？  
```

对时间数据处理完成之后，就可以进行时间维度上的销售情况分析，与**任务1**一致，评定指标也选择为销售数量之和。

In [11]:
#销售数量变化趋势
time_series_df = df.groupby("购药日期").agg({"销售数量": "sum"}).reset_index()
plt.figure(figsize=(15, 7))
sns.lineplot(data=time_series_df, x="购药日期", y="销售数量", color="royalblue")
plt.title("总销售数量变化趋势")
plt.xlabel("日期")
plt.ylabel("总销售量")
plt.tight_layout()
plt.show()

/opt/conda/lib/python3.6/site-packages/pandas/plotting/_converter.py:129: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)


<Figure size 1080x504 with 1 Axes>

In [12]:
#销售额变化趋势
time_series_df = df.groupby("购药日期").agg({"实收金额": "sum"}).reset_index()
plt.figure(figsize=(15, 7))
sns.lineplot(data=time_series_df, x="购药日期", y="实收金额", color="royalblue")
plt.title("销售金额变化趋势")
plt.xlabel("日期")
plt.ylabel("整体销售额")
plt.tight_layout()
plt.show()

<Figure size 1080x504 with 1 Axes>

### 任务3：各月份卖的最好的药品是哪些？

In [13]:
#提取月份
df["月份"] = df["购药日期"].dt.month
monthly_sales = df.groupby(["月份", "商品名称"]).agg({"销售数量": "sum"})
top_medicines_by_month = monthly_sales.groupby("月份")["销售数量"].idxmax()
top_medicines_details = monthly_sales.loc[top_medicines_by_month].reset_index()
top_medicines_details

,月份,商品名称,销售数量
0,1,开博通,267
1,2,苯磺酸氨氯地平片(安内真),193
2,3,开博通,237
3,4,苯磺酸氨氯地平片(安内真),582
4,5,开博通,261
5,6,苯磺酸氨氯地平片(安内真),219
6,7,苯磺酸氨氯地平片(安内真),131


### 任务4：人们是否会更愿意购买可使用社保减免的药品？  

提示：  

数据集中涉及到社保的只有**社保卡号**的信息，所以我们需要根据**应收金额**与**实收金额**的差异来评定药品是否支持社保减免。  

分析思路：  
1. 计算应收金额与实收金额的差  
2. 根据差值进行评定（差值为0表示应收金额与实收金额一致，不支持社保减免；差值不为0则反之）

In [14]:
#增加一个特征，判断是否使用了医保
df["是否使用医保"] = np.where(df["应收金额"] - df["实收金额"] == 0, "否", "是")

**为了判断人们是否会更愿意购买可使用社保减免的药品，按“是否使用医保”特征对数据进行分组，然后计算每组的销售数量总和。这样，我们就可以比较使用医保和不使用医保的药品销售数量，从而判断人们是否更愿意购买可以使用医保减免的药品。**

In [15]:
medicare_sales = df.groupby("是否使用医保").agg({"销售数量": "sum"}).reset_index()
plt.figure(figsize=(8, 6))
sns.barplot(data=medicare_sales, x="是否使用医保", y="销售数量", palette="viridis")
plt.title("医保使用情况")
plt.xlabel("是否使用医保")
plt.ylabel("总量")
plt.tight_layout()
plt.show()

<Figure size 576x432 with 1 Axes>

### 任务5：探究药品的销售数量与时间是否有关

In [16]:
weekday_ordering = {
    '星期一': 1,
    '星期二': 2,
    '星期三': 3,
    '星期四': 4,
    '星期五': 5,
    '星期六': 6,
    '星期日': 7
}

weekly_sales = df.groupby("星期").agg({"销售数量": "mean"}).reset_index()
weekly_sales["weekday_num"] = weekly_sales["星期"].map(weekday_ordering)
weekly_sales = weekly_sales.sort_values(by="weekday_num")

# Plotting the average sales quantity for each weekday
plt.figure(figsize=(12, 6))
sns.barplot(data=weekly_sales, x="星期", y="销售数量", order=weekly_sales["星期"], palette="viridis")
plt.title("按星期划分的平均销售数量")
plt.xlabel("星期")
plt.ylabel("平均销售量")
plt.tight_layout()
plt.show()

<Figure size 864x432 with 1 Axes>

In [17]:
#进行Pearson相关分析
#销售数量与星期之间的相关系数
df["星期_数字格式"] = df["星期"].map(weekday_ordering)
correlation = df[["星期_数字格式", "销售数量"]].corr().iloc[0, 1]
print('销售数量与星期之间的相关系数:',correlation)
#销售数量与月份之间的相关系数
month_correlation = df[["月份", "销售数量"]].corr().iloc[0, 1]
print('销售数量与月份之间的相关系数:',month_correlation)
df["日"] = df["购药日期"].dt.day
day_correlation = df[["日", "销售数量"]].corr().iloc[0, 1]
print('销售数量与日之间的相关系数:',month_correlation)

销售数量与星期之间的相关系数: -0.019199858000298713
销售数量与月份之间的相关系数: 0.006682612133274321
销售数量与日之间的相关系数: 0.006682612133274321


**通过上述操作，通过时序图可以看出来，销售数量存在某种周期规律，但是拆分为月、日、周，却发现销售数量与时间是无关的，工作日和休息日的销售数量也是比较接近的，所以判断药品的销售数量与时间无关。**

### 任务6：探究药品的销售数量与可否使用社保有关

In [18]:
#分组比较
medicare_sales_avg = df.groupby("是否使用医保").agg({"销售数量": "mean"}).reset_index()
plt.figure(figsize=(8, 6))
sns.barplot(data=medicare_sales_avg, x="是否使用医保", y="销售数量", palette="viridis")
plt.title("基于医保使用情况的平均销售数量")
plt.xlabel("是否使用医保")
plt.ylabel("平均销售量")
plt.tight_layout()
plt.show()

<Figure size 576x432 with 1 Axes>

In [19]:
#Pearson相关分析
df["是否使用医保_数字格式"] = df["是否使用医保"].map({"是": 1, "否": 0})
medicare_correlation = df[["是否使用医保_数字格式", "销售数量"]].corr().iloc[0, 1]
print('医保使用情况与销售数量的相关系数:',medicare_correlation)

医保使用情况与销售数量的相关系数: -0.03260330021236449


**虽然使用医保的销售数量远大于未使用医保的药品，但是通过分组比较，探究平均销售数量，以及Pearson相关分析，得出药品的销售数量与可否使用社保其实是没有关系的。**

# 4、总结  

卖的最好的是：苯磺酸氨氯地平片(安内真)，可以继续采购  
卖的最差的是：D厄贝沙坦氢氯噻嗪片(倍悦)、TG厄贝沙坦片、TG盐酸贝那普利片(新亚富舒)，可以放弃采购或减少采购  
药品的销售数量与时间无关。  
药品的销售数量与可否使用社保无关。